In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("/content/gdrive/MyDrive/latihanchatbot/data.json").read()
intents = json.loads(data_file)

In [ ]:
# preprocessing the json data
 #tokenization
nltk.download('punkt')
nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

232 documents
76 classes ['active_prevention', 'app_benefits', 'attracting_beneficial_insects', 'benefits_of_companions', 'benefits_of_crop_rotation', 'benefits_of_organic_approach', 'choosing_the_right_variety', 'common_pests', 'common_sowi_issues', 'companion_planting_benefits', 'companion_plants', 'companion_plants_sawi', 'composting_benefits', 'container_gardening', 'control_methods', 'controlling_disease', 'cost_of_app', 'creating_safe_habitat', 'crop_rotation_benefits', 'dealing_with_common_pests', 'diagnosing_problems', 'environmentally_friendly', 'feedback', 'fertilizing_sowi', 'garden_layout', 'gardening_in_small_spaces', 'greeting', 'growth_phases', 'harvesting_sowi', 'harvesting_tools', 'identifying_specific_pests', 'integrating_technology', 'introduction', 'keeping_soil_healthy', 'main_pests', 'maximizing_yield', 'noanswer', 'nutrient_management', 'nutrition_for_sawi', 'nutritional_deficiencies', 'organic_disease_management', 'organic_fertilizers', 'organic_pest_control', '

In [ ]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]

    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

Training data created


In [ ]:
from tensorflow.keras.optimizers import SGD

# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model with the new SGD optimizer
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)
model.save('models.h5', hist)  # We will pickle this model to use in the future
print("\n")
print("*" * 50)
print("\nModel Created Successfully!")


Epoch 1/500
47/47 [==============================] - 1s 3ms/step - loss: 4.3538 - accuracy: 0.0086
Epoch 2/500
47/47 [==============================] - 0s 3ms/step - loss: 4.3122 - accuracy: 0.0086
Epoch 3/500
47/47 [==============================] - 0s 3ms/step - loss: 4.2733 - accuracy: 0.0345
Epoch 4/500
47/47 [==============================] - 0s 3ms/step - loss: 4.2073 - accuracy: 0.0560
Epoch 5/500
47/47 [==============================] - 0s 3ms/step - loss: 4.1407 - accuracy: 0.0862
Epoch 6/500
47/47 [==============================] - 0s 3ms/step - loss: 4.0818 - accuracy: 0.0905
Epoch 7/500
47/47 [==============================] - 0s 3ms/step - loss: 3.9304 - accuracy: 0.1207
Epoch 8/500
47/47 [==============================] - 0s 3ms/step - loss: 3.7684 - accuracy: 0.1379
Epoch 9/500
47/47 [==============================] - 0s 3ms/step - loss: 3.4879 - accuracy: 0.1940
Epoch 10/500
47/47 [==============================] - 0s 3ms/step - loss: 3.3189 - accuracy: 0.2112
Epoch 11/